# Mass spectrometry data

The objective of this exercise is to read in raw peptide MSMS spectrum information and output a dataframe.
The .msp file can be downloaded [here](https://chemdata.nist.gov/download/peptide_library/libraries/cptaclib/2015/cptac2_mouse_hcd_selected.msp.tar.gz).

The information in this ASCII based text file is organized spectrum by spectrum.
The first line per spectrum provides formatted like this:

&emsp;<code>Name: sequence/charge_nmods_collisionenergy</code>

followed by a comment section which can be disregarded and the actual spectrum data which is tab-separated:

&emsp;<code>m/z&emsp;intensity&emsp;additional_info</code>

Spectra are separated by an empty line.

Code a function that returns two DataFrames or arrays containing the processed and filtered data. The first one should contain the spectrum information (n_spectra, n_m/z_features) and the second one the sequences per row (n_spectra).

Here are some general guidelines:

* The m/z values need to be binned to integer values (mathematically rounded), otherwise the dataframe size would get out of hand. This will allow for multiple values mapped to a single bin (e.g. if there are peaks at 145.1 and 145.2). Here, only the maximum of those peaks should be kept in the final dataframe.

* Rows that are all-zero should be dropped.

Your function should allow for selecting a range on the x-axis (m/z-range). All peaks outside this range can be disregarded. Furthermore, only spectra within a set collision energy range and a maximum sequence length should be contained in the output dataframe.

The faster your function runs, the better. I will time them all in the end.

In [1]:
import numpy as np
import pandas as pd
import timeit
import re
from pathlib import Path

In [2]:
def get_seq_ce_tupel(line):
    sequenz = re.findall(r"(?<=Name: )(.*)(?=/)",line)
    sequenz = sequenz[0]
    collision_energy = re.findall(r"(\d+\.\d+|\d+)eV",line)
    collision_energy = float(collision_energy[0])
    return (sequenz, collision_energy)

def get_line_type(line):
    number = re.findall(r"\d",line[0])
    if len(number) == 1:
        line_type = "data"
    else:
        name = re.findall(r"Name:",line)
        if len(name) == 1:
            line_type = "first"
        else:
            if line == "\n":
                line_type = "last"
            else:
                line_type = "skip"
    return line_type

def get_data_tupel(line):
    data_list = re.findall(r"(\d+\.\d+|\d+)\t", line)
    data_list = [float(l) for l in data_list]
    data_tupel = (int(round(data_list[0])),data_list[1])
    return data_tupel


line = "Name: AAAEEGCGVGVEDDRELEELLESALDDFDK/3_1(6,C,CAM)_50.2eV"
#line = "Name: AAAAAAAAAAEEAAMQRDLLPPAGR/3_0_34.9eV"
#line = "Num peaks: 1189"
#line = "Comment: Single Pep=N-Semitryptic Mods=0 Fullname=A.AAAAAAAAAAEEAAMQRDLLPPAGR.R Charge=3 Parent=788.7411 Se=1(^G1:sc=4.56416e-009) Mz_diff=-9ppm"
#line = '166.0501	2077	"Int/AE-H2O-NH3/1.2ppm,Int/EA-H2O-NH3/1.2ppm"'
#print(get_data_tupel(line))
print(get_seq_ce_tupel(line))

('AAAEEGCGVGVEDDRELEELLESALDDFDK', 50.2)


In [3]:


def msp_to_df(
    input_file,
    max_seq_len=30,
    min_ce=36,
    max_ce=40,
    mz_min=135,
    mz_max=1400,
):
    """
    Function to read spectrum data from .msp file and convert to dataframe.
    Args:
        input_file (str): path to .msp file
        max_seq_len (int): maximum acceptable sequence length
        min_ce (int): minimum collision energy of spectra to be included in df
        max_ce (int): maximum collision energy of spectra to be included in df
        mz_min (int): lower boundary for m/z to be included in df
        mz_max (int): lower boundary for m/z to be included in df

    Returns:
        df (pd.DataFrame or np.array): spectrum information within defined parameters [n_spectra, n_features]
        seqs (pd.DataFrame or np.array): sequences
    
    """
    data_dict_list = []
    sequenz_dict_list = []

   
    with open(input_file) as input_data:
        
        for line in input_data:

            line_type = get_line_type(line)

            if line_type == "first":
                #Inizialize values
                data_line_dict = {}
                previous_tuple =("nothing","in here")
                current_bin = mz_min
                skip_sequenz = False
                
                #Check if sequenz meets the requirements
                seq_ce_tupel = get_seq_ce_tupel(line)
                condition_seq_len = len(seq_ce_tupel[0]) <= max_seq_len
                condition_ce = min_ce <= seq_ce_tupel[1] <= max_ce
                if not(condition_seq_len and condition_ce):
                    skip_sequenz = True
                    continue
                
                sequenz_dict_list.append({"Sequenz":seq_ce_tupel[0]})
                
                
            if line_type == "data" and skip_sequenz == False:

                data_tuple = get_data_tupel(line)
                
                if data_tuple[0] == previous_tuple[0]:
                    #print("if")
                    current_bin -= 1
                    compare_list = [data_tuple[1],previous_tuple[1]]
                    compare_list.sort()
                    

                    if data_tuple[0] < mz_min:
                        current_bin += 1
                        previous_tuple = data_tuple  
                        continue
                    if data_tuple[0] > mz_max:
                        current_bin += 1
                        previous_tuple = data_tuple  
                        continue
                    if data_tuple[0] > current_bin:
                        while data_tuple[0] > current_bin:
                            data_line_dict[current_bin] = np.nan
                            current_bin += 1
                    if data_tuple[0] == current_bin:                                           
                        data_line_dict[current_bin] = compare_list[1]
                        current_bin += 1

                else:
                    #print("else")
                    if data_tuple[0] < mz_min:
                        previous_tuple = data_tuple  
                        continue
                    if data_tuple[0] > mz_max:
                        previous_tuple = data_tuple  
                        continue
                    if data_tuple[0] > current_bin:
                        while data_tuple[0] > current_bin:
                            data_line_dict[current_bin] = np.nan
                            current_bin += 1
                    if data_tuple[0] == current_bin:                                           
                        data_line_dict[current_bin] = data_tuple[1]
                        current_bin += 1
                
                previous_tuple = data_tuple    
                

                                
            if line_type == "skip":
                continue
            if line_type == "last" and skip_sequenz == False:
                if current_bin <= mz_max:
                    while current_bin <= mz_max:
                        data_line_dict[current_bin] = np.nan
                        current_bin += 1
                if len(data_dict_list) == -5:

                    print(data_line_dict)
                data_dict_list.append(data_line_dict)


                
    df_final = pd.DataFrame.from_dict(data_dict_list)
    seq_final = pd.DataFrame.from_dict(sequenz_dict_list)

    df_final = df_final.fillna(0)

    df_final = df_final.loc[~(df_final==0).all(axis=1)]
    seq_final = seq_final.loc[~(df_final==0).all(axis=1)]

    return (df_final, seq_final)

In [5]:
input_file = Path("..\..\..\cptac2_mouse_hcd_selected.msp")
results = msp_to_df(input_file)
results

(      135      136     137   138     139   140      141     142      143   \
 0      0.0   2851.7     0.0   0.0     0.0   0.0   1404.0     0.0  26365.8   
 1      0.0   2626.0     0.0   0.0  1553.4   0.0      0.0     0.0  11172.9   
 2      0.0  59857.3  8295.8   0.0  3220.5   0.0  24351.6  2054.0  14686.4   
 3      0.0      0.0     0.0   0.0     0.0   0.0      0.0     0.0      0.0   
 4      0.0      0.0     0.0   0.0     0.0   0.0      0.0     0.0      0.0   
 ...    ...      ...     ...   ...     ...   ...      ...     ...      ...   
 2128   0.0      0.0     0.0   0.0     0.0   0.0      0.0     0.0      0.0   
 2129   0.0  51469.1  2589.3   0.0     0.0   0.0      0.0     0.0      0.0   
 2130   0.0      0.0     0.0   0.0     0.0   0.0      0.0     0.0  42959.4   
 2131   0.0      0.0     0.0   0.0     0.0   0.0      0.0     0.0   3538.1   
 2132   0.0  31606.0  1451.6   0.0     0.0   0.0      0.0     0.0      0.0   
 
          144   ...  1391  1392  1393  1394  1395  1396    139